In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas
  
!unzip /content/drive/MyDrive/datasets/data.zip

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# extract zip file
import zipfile
import os
train = '/content/ready/train'
val = '/content/ready/val'

In [ ]:
# image augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 15,
    featurewise_center=True,
    brightness_range=(0.1,0.9),
)

validation_datagen = ImageDataGenerator(
      rescale = 1.0/255,
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train,
    target_size = (50, 50),
    color_mode = "grayscale",
    class_mode = 'categorical',
    batch_size = 128
)

validation_generator = validation_datagen.flow_from_directory(
    val,
    target_size = (50, 50),
    color_mode = "grayscale",
    class_mode = 'categorical',
    batch_size = 128
)

Found 7061 images belonging to 4 classes.
Found 1958 images belonging to 4 classes.


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (50, 50, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.54),

    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'), 
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.7),

    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.7),

    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.6),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.64),

    tf.keras.layers.Dense(4, activation = 'softmax')
])

In [ ]:
# count loss function and optimizer
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = "rmsprop",
    metrics = ['accuracy']               
)

In [ ]:
# callbacks
accuracy_threshold = 98e-2
class my_callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if logs.get('accuracy') >= accuracy_threshold:
            print('\nFor Epoch', epoch, '\nAccuracy has reach = %2.2f%%' %(logs['accuracy']*100), 'training has been stopped.')
            self.model.stop_training = True

In [ ]:
# train data
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.n//train_generator.batch_size,
    epochs = 150,
    validation_data = validation_generator,
    validation_steps = validation_generator.n//validation_generator.batch_size
)

In [ ]:
import numpy as np
from google.colab import files
import keras.utils as image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
%matplotlib inline

# predict images
path = "/content/ready/val/paper/0.png"

img_source = Image.open(path).convert('L').resize((150,100))
imgplot = plt.imshow(img_source, cmap=plt.get_cmap('gray'))
x = image.img_to_array(img_source)
x = np.expand_dims(x, axis = 0)

images = np.vstack([x])
classes = model.predict(images, batch_size = 10)

print(path)
if classes[0, 0] == 1:
  print('paper')
elif classes[0, 1] == 1:
  print('rock')
elif classes[0, 2] == 1:
  print('scissors')

In [ ]:
model_json = model.to_json()

with open("model1/model.json","w") as model_json_file:
    model_json_file.write(model_json)
    
model.save_weights("model1/model_weights.h5")
print("model saved")

model saved
